In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif api_key[:8]!="sk-proj-":
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip()!= api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str
    
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library 
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [7]:
system_prompt = "You are an assistant the analyzes the contents of a website and provides a short summary,\
ignoring text that might be navigation related. Respond is markdown."

In [13]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
system_prompt

'You are an assistant the analyzes the contents of a website and provides a short summary,ignoring text that might be navigation related. Respond is markdown.'

In [14]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [16]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
print(messages_for(ed))

[{'role': 'system', 'content': 'You are an assistant the analyzes the contents of a website and provides a short summary,ignoring text that might be navigation related. Respond is markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massiv

In [23]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [24]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a platform for his professional endeavors and personal interests. He is a software developer with a passion for coding and experimenting with large language models (LLMs). In addition to his tech-related pursuits, he enjoys DJing, electronic music production, and engaging with the tech community through platforms like Hacker News.\n\n## Professional Background\n- **Co-founder & CTO of Nebula.io**: Focused on leveraging AI to help individuals identify their potential and engage effectively with talent.\n- **Former Founder & CEO of untapt**: An AI startup acquired in 2021, which has received notable media coverage and has a patented matching model for talent sourcing.\n\n## Key Announcements\n- **September 15, 2025**: "AI in Production: Gen AI and Agentic AI on AWS at scale"\n- **May 28, 2025**: "Connecting my courses – become an LLM expert and leader"\n- **May 18, 2025**: "2025 AI Executive Briefing"\n- **April

In [25]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [26]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal hub where he shares his interests in coding, experimenting with large language models (LLMs), and his passion for electronic music. He is the co-founder and CTO of Nebula.io, which applies AI to help individuals discover their potential in the talent management field. Previously, he founded an AI startup called untapt, which was acquired in 2021. 

**Key Offerings:**
- Insights into LLM technology and applications.
- Courses designed to help individuals become experts in LLMs.

**Recent Announcements:**
- **September 15, 2025:** Discussion on AI in production, focusing on Gen AI and Agentic AI at scale on AWS.
- **May 28, 2025:** Offering courses to become an expert and leader in LLMs.
- **May 18, 2025:** Announcement for the 2025 AI Executive Briefing.
- **April 21, 2025:** Launch of the "Complete Agentic AI Engineering Course".

For connectivity, Edward encourages visitors to reach out and subscribe to his newsletter for updates.

In [27]:
display_summary("https://cnn.com")

# Summary of CNN Website Content

CNN is dedicated to providing up-to-the-minute news and analysis across a wide variety of topics including US and world news, politics, business, health, entertainment, science, and more. The site adopts an interactive and feedback-oriented approach, valuing user input regarding advertisements and technical issues.

### Key News Highlights:
- **Palestinian Statehood Recognition**: Palestinians express skepticism about international statehood recognition, with an Israeli response indicating intentions to counter these moves.
  
- **Tylenol and Autism**: A US Department of Health and Human Services report suggests a potential link between the use of Tylenol during pregnancy and autism, spotlighting concerns regarding causation versus correlation. Dr. Sanjay Gupta provided insights on this point.

- **Rural Health Clinics Closure**: Following the implementation of a significant healthcare legislation by President Trump, several rural health clinics have announced closures, indicating possible political ramifications.

- **International Affairs**: Current global events include investigations into Russian military activity over NATO territories, the impact of climate-related disruptions in Europe, and military exercises in Venezuela as a show of strength against the US.

- **Cultural and Recreational Updates**: Entertainment highlights feature celebrities such as Harry Styles running marathons and discussions about significant cultural shifts impacting drag culture amid current events.

The platform encompasses various features including videos, live TV, and a range of podcasts to keep users engaged and informed on the latest developments worldwide.

In [28]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is a public benefit corporation focused on developing AI technologies that prioritize safety and human benefits. The company emphasizes responsible AI development through tools like Claude, its AI product line, which includes Claude Opus 4.1, Claude Sonnet 4, and Claude Haiku 3.5. 

## Key Features and Offerings:
- **Claude Products**: A family of AI models designed for various applications including coding, customer support, and education. 
- **API Access**: Developers can build applications using Claude's API with comprehensive documentation.
- **Customer Collaborations**: Solutions tailored to various sectors like financial services, government, and the education sector.
- **Learning Resources**: Anthropic Academy provides educational material on how to effectively use Claude.

## Recent News and Announcements:
- **ISO 42001 Certification**: Announced as part of their commitment to quality standards in AI development.
- **Claude Sonnet 4 and Claude Opus 4.1**: Latest models announced with extended functionalities for users.
- **Anthropic Economic Index**: A research initiative aimed at understanding AI’s impact on the labor market, providing reports and data for public access.

The website further includes a commitment to transparency, responsible scaling policies, and insights into AI safety, reinforcing Anthropic's mission to ensure the positive impact of AI on society.